# Problem Statement
Predicting turbine energy yield (TEY) using ambient variables as features.

# Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('gas_turbines.csv')
df.head()

# Exploratory Data Analysis

In [ ]:
df.isnull().sum()

### Descriptive Analysis

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.corr()["TEY"].sort_values(ascending=False)

In [ ]:
numerical_features = df.describe(include=["int64","float64"]).columns
numerical_features

### Data Visualization

### Univariate plots

In [ ]:
numerical_features=[feature for feature in df.columns if df[feature].dtypes != 'O']
for feat in numerical_features:
    skew = df[feat].skew()
    sns.distplot(df[feat], kde= False, label='Skew = %.3f' %(skew), bins=30)
    plt.legend(loc='best')
    plt.show()

In [ ]:
plt.figure()
sns.histplot(data=df,x="TEY",color="red",kde=True)
plt.axvline(x=df["TEY"].mean(),ymax=0.55,color="green",linestyle='--',label="Mean")
plt.axvline(x=df["TEY"].median(),ymax=0.56,color="purple",linestyle='--',label="Median")
plt.legend()
plt.title("Histogram of the Target Column")

###### Unsurprisingly, Mostly none of the features are on the same scale as we already saw in the previous section.

### Multivariate Analysis

In [ ]:
sns.pairplot(df,
                 markers="+",
                 kind='reg',
                 diag_kind="auto",
                 plot_kws={'line_kws':{'color':'#aec6cf'},
                           'scatter_kws': {'alpha': 0.5,
                                           'color': '#82ad32'}},
               
                 diag_kws= {'color': '#82ad32'})

In [ ]:
x = df.drop('TEY', axis=1)
y = df[["TEY"]]

# Feature Selection Technique

In [ ]:
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [ ]:
test = SelectKBest(score_func = mutual_info_regression, k ='all')
fit = test.fit(x, y)

In [ ]:
scores = fit.scores_
features = fit.transform(x)

In [ ]:
score_df = pd.DataFrame(list(zip(scores, x.columns)), columns = ['Score', 'Feature'])
score_df.sort_values(by ="Score", ascending = False, inplace = True)
score_df

In [ ]:
fig, axes = plt.subplots(figsize = (20, 6))
plt.bar([i for i in range(len(scores))], scores)
axes.set_xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
axes.set_xticklabels(x.columns.values)
plt.xticks(rotation = 90, size = 15)
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
sns.barplot(x ='Feature', y ="Score", data = score_df, order = score_df.sort_values('Score').Feature)
plt.xlabel("Features", size=15)
plt.ylabel("Scores", size=15)
plt.yticks(rotation = 0, fontsize = 14)
plt.xticks(rotation = 90, fontsize = 16)
plt.title("Feature Score w.r.t the Sales", size=18)
plt.show()

In [ ]:
score_df.sort_values('Score', ascending = False)

In [ ]:
model_data = df[['CDP', 'GTEP','TIT', 'TAT', 'AFDP', 'CO', 'AT',"TEY"]]
model_data.head()

### 5.1. Data Pre-Processing
##### Feature Engineering

In [ ]:
continuous_feature=[feature for feature in model_data.columns if model_data[feature].dtype !='O']
print('Continuous Feature Count {}'.format(len(continuous_feature)))

In [ ]:
df_standard_scaled = model_data.copy()
features = df_standard_scaled[continuous_feature]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df_standard_scaled[continuous_feature] = scaler.fit_transform(features.values)
df_standard_scaled.head()

### Test Train Split With Imbalanced Dataset

In [ ]:
x = df_standard_scaled.drop('TEY',axis=1)
y = df_standard_scaled[['TEY']]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)

# Hyperparameter Tuning

### Hyperparameter Tuning: Optimal Learning rate ,Number of Layers and Neurons

In [ ]:
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers

def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=100, step=32), activation = 'relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                  loss ='mean_absolute_error', metrics =['mean_absolute_error'])
    return model

In [ ]:
!pip install kerastuner
!pip install --upgrade pip


from kerastuner.tuners import RandomSearch
tuner = RandomSearch(build_model, objective ='val_mean_absolute_error', max_trials = 5, executions_per_trial = 3,
                     directory ='project', project_name ='Gas Turbine')

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(x_train, y_train, epochs = 100, validation_data = (x_test, y_test))

In [ ]:
tuner.results_summary()

### Hyperparameter Tuning: Optimal Batch_size, Number of Epochs

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold
from keras.layers import Dense
from keras.optimizers import Adam

def create_model():
    model1 = Sequential()
    model1.add(Dense(32, input_dim = 7, activation ='relu'))
    model1.add(Dense(64, activation ='relu'))
    model1.add(Dense(96, activation ="relu"))
    model1.add(Dense(32, activation ="relu"))
    model1.add(Dense(64, activation ="relu"))
    model1.add(Dense(32, activation ="relu"))
    model1.add(Dense(96, activation ="relu"))
    model1.add(Dense(96, activation ="relu"))
    model1.add(Dense(32, activation ="relu"))
    model1.add(Dense(64, activation ="relu"))
    model1.add(Dense(64, activation ="relu"))
    model1.add(Dense(units = 1, activation ="linear"))
    adam = Adam(learning_rate=0.001)
    model1.compile(loss ='mean_absolute_error', optimizer = adam, metrics = ["mean_absolute_error"])
    return model1

In [ ]:
model1 = KerasRegressor(build_fn = create_model, verbose = 0)
batch_size = [10, 20, 40, 50]
epochs = [10, 50, 100, 200]
param_grid = dict(batch_size = batch_size, epochs = epochs)
grid = GridSearchCV(estimator = model1, param_grid = param_grid, cv = KFold(), verbose = 10)

In [ ]:
grid_result = grid.fit(x_test, y_test)

In [ ]:
print('Best {}, using {}'.format(grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]
for mean, stdev, param in zip(means, stds, params):
    print("{},{} with {}".format(mean, stdev, param))

### Hyperparameter Tuning: Optimal Droupout rate

In [ ]:
from keras.layers import Dropout

def create_model(dropout_rate):
    model2 = Sequential()
    model2.add(Dense(32, input_dim = 7, activation ='relu'))
    model2.add(Dense(64, activation ='relu'))
    model2.add(Dense(96, activation ="relu"))
    model2.add(Dense(32, activation ="relu"))
    model2.add(Dense(64, activation ="relu"))
    model2.add(Dense(32, activation ="relu"))
    model2.add(Dense(96, activation ="relu"))
    model2.add(Dense(96, activation ="relu"))
    model2.add(Dense(32, activation ="relu"))
    model2.add(Dense(64, activation ="relu"))
    model2.add(Dense(64, activation ="relu"))
    model2.add(Dense(units = 1, activation ="linear"))
    adam = Adam(learning_rate = 0.001)
    model2.compile(loss ='mean_absolute_error', optimizer = adam, metrics = ["mean_absolute_error"])
    return model2

In [ ]:
model2 = KerasRegressor(build_fn = create_model, batch_size = 40, epochs = 200, verbose = 0)
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
param_grid = dict(dropout_rate = dropout_rate)
grid2 = GridSearchCV(estimator = model2, param_grid = param_grid, cv = KFold(), verbose = 10)

In [ ]:
grid_result2 = grid2.fit(x_test, y_test)

In [ ]:
print('Best {}, using {}'.format(grid_result2.best_score_, grid_result2.best_params_))
means = grid_result2.cv_results_["mean_test_score"]
stds = grid_result2.cv_results_["std_test_score"]
params = grid_result2.cv_results_["params"]
for mean, stdev, param in zip(means, stds, params):
    print("{},{} with {}".format(mean, stdev, param))

### Hyperparameter Tuning: Optimal Activation Function and Kernel Initializer

In [ ]:
def create_model(activation_function, init):
    model3 = Sequential()
    model3.add(Dense(32, input_dim = 7, activation ='relu'))
    model3.add(Dropout(0.3))
    model3.add(Dense(64, activation ='relu'))
    model3.add(Dropout(0.3))
    model3.add(Dense(96, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(32, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(64, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(32, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(96, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(96, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(32, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(64, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(64, activation ="relu"))
    model3.add(Dropout(0.3))
    model3.add(Dense(units = 1, activation ="linear"))
    
    adam=Adam(learning_rate = 0.001)
    model3.compile(loss ='mean_absolute_error', optimizer = adam, metrics = ["mean_absolute_error"])
    return model3

In [ ]:
model3 = KerasRegressor(build_fn = create_model, batch_size = 40, epochs = 200, verbose = 0)
activation_function = ['softmax','tanh','relu']
init = ['normal','uniform','zero']
param_grid = dict(activation_function = activation_function, init = init)
grid3 = GridSearchCV(estimator = model3, param_grid = param_grid, cv = KFold(), verbose = 10)

In [ ]:
grid_result3 = grid3.fit(x_test, y_test)

In [ ]:
print('Best {}, using {}'.format(grid_result3.best_score_, grid_result3.best_params_))
means = grid_result3.cv_results_["mean_test_score"]
stds = grid_result3.cv_results_["std_test_score"]
params = grid_result3.cv_results_["params"]
for mean, stdev, param in zip(means, stds, params):
    print("{},{} with {}".format(mean, stdev, param))

# Model Building Neural Networks

In [ ]:
model_data.head()

In [ ]:
x = model_data.drop('TEY', axis = 1)
y = model_data[["TEY"]]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)

scaler_train = StandardScaler()
scaler_test = StandardScaler()

x_train_scaled = scaler_train.fit_transform(x_train) 
x_test_scaled  = scaler_test.fit_transform(x_test) 

print(x_train_scaled.shape)
print(x_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
model = Sequential()
model.add(Dense(units = 50 , activation = 'relu', kernel_initializer = 'normal', input_dim = 7))
model.add(Dense(units = 20 , activation = 'tanh', kernel_initializer = 'normal')) 
model.add(Dense(units = 1  , kernel_initializer = 'normal')) 

model.compile(optimizer = "adam", loss ="mse", metrics = ["mae", "mse"])
history = model.fit(x_train_scaled, y_train , batch_size = 50, validation_split = 0.3, epochs = 100,  verbose = 1)

In [ ]:
def toFindBestParams(x_train_scaled, y_train, x_test_scaled, y_test):
    batch_size_list = [5 , 10 , 15 , 20]
    epoch_list      = [5 , 10 , 50 , 100]
    bestParamTable = pd.DataFrame()
    
    for batch_trial in batch_size_list:
        for epochs_trial in epoch_list:
            model = Sequential()
            model.add(Dense(units = 50, input_dim = x_train_scaled.shape[1], kernel_initializer ='normal', activation ='relu'))
            model.add(Dense(units = 20, kernel_initializer ='normal', activation ='tanh'))
            model.add(Dense(1, kernel_initializer ='normal'))
            model.compile(optimizer ='adam', loss = 'mean_squared_error')
            model.fit(x_train_scaled, y_train , batch_size = batch_trial, epochs = epochs_trial,  verbose = 0)
            MAPE = np.mean(100 * (np.abs(y_test - model.predict(x_test_scaled))/ y_test))  
            bestParamTable = bestParamTable.append(pd.DataFrame(data = [[batch_trial, epochs_trial, 100 - MAPE]],
                                                        columns = ['batchsize','epochs','Accuracy'] ))
            print('batch_size:', batch_trial,'-', 'epochs:',epochs_trial, 'Accuracy:',100-MAPE)
    return bestParamTable

finalParamTable = toFindBestParams(x_train_scaled, y_train, x_test_scaled, y_test)
finalParamTable

In [ ]:
finalParamTable = finalParamTable.reset_index()

#### Training Model - using best params

In [ ]:
model.compile(optimizer ='adam', loss = 'mean_squared_error')
model.fit(x_train_scaled, y_train, batch_size = 20 , epochs = 100, verbose = 0)

### Predicting values from Model using same dataset

In [ ]:
y_predict_test = model.predict(x_test_scaled) 
predictions_df = pd.DataFrame(x_test)
predictions_df['Actual'] = y_test
predictions_df['Predicted'] = y_predict_test
print(predictions_df.shape)
predictions_df.head(10)

In [ ]:
predictions_df.drop(['CDP','GTEP','TIT','TAT','AFDP','CO','AT'], axis = 1, inplace = True)

### Calculating Absolute Percent Error and Error

In [ ]:
# Computing the absolute percent error
APE=100*(abs(predictions_df['Actual'] - predictions_df['Predicted'])/predictions_df['Actual'])
print('The Accuracy for Test Data -- ANN model = ', 100-np.mean(APE))

# adding absolute percent error to table
predictions_df['APE %'] = APE
predictions_df.head()

In [ ]:
predictions_df['Error'] = (predictions_df['Actual'] - predictions_df['Predicted'])/(predictions_df['Actual'])
predictions_df.reset_index(drop = True)

### Visualizing the Relationship between the Actual and Predicted ValuesModel Validation

In [ ]:
plt.figure(figsize = (12,8))
plt.xlabel("Actual Values")
plt.ylabel("Predicted values")
plt.title("The Scatterplot of Relationship between Actual Values and Predictions")
plt.scatter(predictions_df['Actual'], predictions_df['Predicted'])

In [ ]:
# We will evaluate our model performance by calculating the residual sum of squares and the explained variance score
from sklearn import metrics
print("MAE:", metrics.mean_absolute_error(y_test, y_predict_test))
print ("MSE:", metrics.mean_squared_error(y_test, y_predict_test))
print("RMSE:", np.sqrt(metrics.mean_squared_error(y_test, y_predict_test)))

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_predict_test = model.predict(x_test_scaled)
y_predict_train = model.predict(x_train_scaled) 
print('R2_score (train): ', r2_score(y_train, y_predict_train))
print('R2_score (test): ', r2_score(y_test, y_predict_test))

In [ ]:
#Evaluation of  the explained variance score (R^2)
print('This shows our model predict % {} of the target correctly'.format(np.round(metrics.explained_variance_score(y_test,y_predict_test)*100,2))) 

### Residual Analysis

In [ ]:
plt.figure(figsize=(12,10))
sns.distplot(y_test - y_predict_test, bins = 50) 

In [ ]:
import statsmodels.api as smf
smf.qqplot(predictions_df['Error'], line = 'q')
plt.title('Normal Q-Q plot of residuals')
plt.show()